In [1]:
from tqdm.notebook import tnrange
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from IPython.display import clear_output
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
import os
import re

In [2]:
plaintext=np.load('present_plain1.npy')
plaintext

array([[239,  77, 239, ..., 157,  71,  38],
       [213, 249,  33, ..., 147,  54, 250],
       [ 59,  31, 150, ..., 241,   7, 255],
       ...,
       [213,  78,  16, ...,  38, 106,  38],
       [167,   7, 136, ..., 184,   1, 220],
       [143,  53, 100, ...,  83, 193,  95]], dtype=uint8)

In [3]:
ciphertext=np.load('present_cipher1.npy')
ciphertext

array([[167,  12, 155, ..., 239,  35, 123],
       [212, 107,  60, ..., 178,  21, 149],
       [164, 193, 219, ..., 188,  79,  64],
       ...,
       [ 38, 224, 127, ..., 206, 190, 210],
       [180,  94, 159, ...,  21,  32, 173],
       [224, 228, 204, ..., 229,  49, 115]], dtype=uint8)

In [4]:
traces=np.load('present_traces1.npy')
traces

array([[ 0.04394531, -0.09179688, -0.05859375, ..., -0.08300781,
         0.03710938,  0.09667969],
       [ 0.04394531, -0.09277344, -0.05957031, ..., -0.0859375 ,
         0.03808594,  0.1015625 ],
       [ 0.04492188, -0.09179688, -0.05761719, ..., -0.08203125,
         0.0390625 ,  0.09863281],
       ...,
       [ 0.04199219, -0.09277344, -0.06054688, ..., -0.08203125,
         0.0390625 ,  0.10058594],
       [ 0.04492188, -0.09179688, -0.05957031, ..., -0.08300781,
         0.04101562,  0.09863281],
       [ 0.04394531, -0.09179688, -0.05957031, ..., -0.08203125,
         0.03808594,  0.09960938]])

In [5]:
plaintext.shape

(15000, 8)

In [6]:
traces.shape

(15000, 5000)

In [7]:
import numpy as np

# PRESENT S-box
PRESENT_SBOX = np.array([
    0xC, 0x5, 0x6, 0xB, 0x9, 0x0, 0xA, 0xD,
    0x3, 0xE, 0xF, 0x8, 0x4, 0x7, 0x1, 0x2
], dtype=np.uint8)

def hamming_weight(x):
    return bin(x).count("1")

def extract_nibbles(plaintexts):
    """Convert (N,8) byte array to (N,16) nibble array (MSB-first)"""
    nibbles = np.zeros((plaintexts.shape[0], 16), dtype=np.uint8)
    
    for i in range(plaintexts.shape[0]):
        for byte_idx in range(8):
            # Extract high nibble (bits 7-4) first
            nibbles[i, byte_idx*2] = (plaintexts[i, byte_idx] >> 4) & 0x0F
            # Then low nibble (bits 3-0)
            nibbles[i, byte_idx*2 + 1] = plaintexts[i, byte_idx] & 0x0F
            
    return nibbles

def cpa_present(traces, plaintexts, start=0, end=5000):
    num_traces, num_points = traces.shape
    plaintext_nibbles = extract_nibbles(plaintexts)
    trace_window = traces[:, start:end].astype(np.float32)
    trace_means = np.mean(trace_window, axis=0)
    trace_stds = np.std(trace_window, axis=0)
    best_key = np.zeros(16, dtype=np.uint8)
    
    for nib_idx in range(16):
        max_corrs = np.zeros(16)
        
        for key_guess in range(16):
            hyp = np.array([
                hamming_weight(PRESENT_SBOX[pt ^ key_guess])
                for pt in plaintext_nibbles[:, nib_idx]
            ], dtype=np.float32)
            
            hyp_mean = np.mean(hyp)
            hyp_std = np.std(hyp)
            
            # FIX: Reshape hyp for proper broadcasting
            hyp_reshaped = (hyp - hyp_mean).reshape(-1, 1)  # Now (15000, 1)
            
            # Now compatible with (15000,5000) traces
            cov = np.mean(hyp_reshaped * (trace_window - trace_means), axis=0)
            corr = cov / (hyp_std * trace_stds)
            
            max_corrs[key_guess] = np.max(np.abs(corr))
        
        best_guess = np.argmax(max_corrs)
        best_key[nib_idx] = best_guess
        print(f"Nibble {nib_idx:2d}: Best guess = {best_guess:1X} (corr {max_corrs[best_guess]:.4f})")
    
    return best_key


In [8]:
key = cpa_present(traces, plaintext)

Nibble  0: Best guess = 0 (corr 0.6773)
Nibble  1: Best guess = 1 (corr 0.6886)
Nibble  2: Best guess = 0 (corr 0.6607)
Nibble  3: Best guess = 1 (corr 0.6448)
Nibble  4: Best guess = 0 (corr 0.6531)
Nibble  5: Best guess = 1 (corr 0.6613)
Nibble  6: Best guess = 0 (corr 0.6547)
Nibble  7: Best guess = 1 (corr 0.6687)
Nibble  8: Best guess = 0 (corr 0.6451)
Nibble  9: Best guess = 1 (corr 0.6792)
Nibble 10: Best guess = 0 (corr 0.6330)
Nibble 11: Best guess = 1 (corr 0.6794)
Nibble 12: Best guess = 0 (corr 0.6384)
Nibble 13: Best guess = 1 (corr 0.6926)
Nibble 14: Best guess = 0 (corr 0.6468)
Nibble 15: Best guess = 1 (corr 0.7033)
